In [5]:
from flask import Flask, request, render_template
import joblib
import pandas as pd
import threading

# Create the Flask app
app = Flask(__name__)

# Load the saved model
model = joblib.load('best_rf_model.pkl')

# Define a route for the home page
@app.route('/')
def home():
    return render_template('index.html')

# Define a route for making predictions
@app.route('/predict', methods=['POST'])
def predict():
    # Get data from the form
    data31 = request.form.to_dict()
    data_df = pd.DataFrame([data31])

    # Define the expected feature names
    expected_columns = [
        'area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms', 'Balcony',
        'parking', 'Lift', 'Price_sqft', 'Furnished', 'Semi-Furnished',
        'Unfurnished', 'New Property', 'Resale', 'Flat', 'Individual House'
    ]

    # Convert data types to numeric
    data_df = data_df.apply(pd.to_numeric, errors='coerce')

    # Ensure all expected columns are present
    for col in expected_columns:
        if col not in data_df.columns:
            data_df[col] = 0  # Default value for missing columns

    # Reorder columns to match the trained model's expected order
    data_df = data_df[expected_columns]

    # Make a prediction using the loaded model
    prediction = model.predict(data_df)

    # Print the prediction for debugging
    print(f'Predicted House Price: ${prediction[0]:,.2f}')

    # Return the prediction
    return render_template('index.html', prediction_text=f'Predicted House Price: ${prediction[0]:,.2f}')

def run_app():
    app.run(debug=True, use_reloader=False)

# Start the Flask app in a background thread
thread = threading.Thread(target=run_app)
thread.start()



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
